In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv('expression_count_test_20240807.csv')
print('Actual df')
print(df)



labels = df.iloc[-1, 1:].tolist() #storing labels in a diffrent variable
df = df.iloc[:-1]   #removing last row as it is label
df = df[df.iloc[:, 1:].ne(0).any(axis=1)]  #removing completely zero rows

print('After removing all zeros rows')
print(df)

print('Labels list')
print(labels)





Actual df
              gene_id  Sample1  Sample2  Sample3  Sample4  Sample5
0       hsa-let-7a-5p   152768   147753    33377   119552    92872
1       hsa-let-7a-3p        2        1        6        1        3
2     hsa-let-7a-2-3p        0        0        0        0        0
3       hsa-let-7b-5p    39937    27455     7200    28693    36428
4       hsa-let-7b-3p        7        6        2       11        8
...               ...      ...      ...      ...      ...      ...
2703    hsa-miR-12132        0        0        0        0        0
2704    hsa-miR-12133        0        0        0        0        0
2705    hsa-miR-12135       59       89       72       48       37
2706    hsa-miR-12136      130      198       42      153       87
2707                *        0        0        0        0        0

[2708 rows x 6 columns]
After removing all zeros rows
            gene_id  Sample1  Sample2  Sample3  Sample4  Sample5
0     hsa-let-7a-5p   152768   147753    33377   119552    92872
1

In [18]:
gene_ids = [
    'hsa-miR-6717-5p', 'hsa-miR-6836-3p', 'hsa-miR-17-3p',
    'hsa-miR-125a-3p', 'hsa-miR-4652-5p', 'hsa-miR-1307-3p',
    'hsa-miR-3131', 'hsa-miR-4534', 'hsa-miR-629-3p',
    'hsa-miR-1185-1-3p', 'hsa-miR-221-3p', 'hsa-miR-4456',
    'hsa-miR-4730', 'hsa-miR-3160-5p', 'hsa-miR-4787-3p',
    'hsa-miR-195-5p', 'hsa-miR-4783-3p', 'hsa-miR-1343-3p',
    'hsa-miR-1185-2-3p', 'hsa-miR-6736-5p'
]

# Filter the DataFrame to get rows with the specified gene_ids
filtered_df = df[df['gene_id'].isin(gene_ids)]
filtered_df = filtered_df.set_index('gene_id').reindex(gene_ids).reset_index()
# filtered_df = filtered_df[:, 1:]
# filtered_df = filtered_df.drop(filtered_df.columns[0], axis=1)
# Display the filtered DataFrame
print(filtered_df)
filtered_df_T = filtered_df.T


filtered_df_T.columns = filtered_df_T.iloc[0]
filtered_df_T = filtered_df_T[1:]
print(filtered_df_T)

              gene_id  Sample1  Sample2  Sample3  Sample4  Sample5
0     hsa-miR-6717-5p      0.0      1.0      0.0      0.0      0.0
1     hsa-miR-6836-3p      4.0      3.0      4.0      4.0      7.0
2       hsa-miR-17-3p    436.0    513.0    188.0    343.0    440.0
3     hsa-miR-125a-3p     36.0      8.0      9.0     52.0     30.0
4     hsa-miR-4652-5p      NaN      NaN      NaN      NaN      NaN
5     hsa-miR-1307-3p    202.0    203.0     94.0    202.0    194.0
6        hsa-miR-3131      2.0      0.0      5.0      0.0      1.0
7        hsa-miR-4534      NaN      NaN      NaN      NaN      NaN
8      hsa-miR-629-3p      2.0      4.0      3.0      1.0      3.0
9   hsa-miR-1185-1-3p    252.0    412.0     31.0    154.0     66.0
10     hsa-miR-221-3p   4299.0   5700.0   1846.0   3306.0   1887.0
11       hsa-miR-4456      NaN      NaN      NaN      NaN      NaN
12       hsa-miR-4730      0.0      1.0      0.0      1.0      0.0
13    hsa-miR-3160-5p      2.0      0.0      0.0      0.0     

In [56]:
normalized_df = filtered_df_T.apply(lambda x: 10 * (x - x.min()) / (x.max() - x.min()), axis=1)  #each row normalizing seperately


normalized_df.columns = normalized_df.iloc[0]
normalized_df = normalized_df[1:]
# normalized_df.reset_index(drop=True, inplace=True)

header_row = pd.DataFrame([normalized_df.columns.tolist()], columns=normalized_df.columns)

# Append the header row to the DataFrame
df_with_header_as_row = pd.concat([header_row, normalized_df], ignore_index=True)
df_with_header_as_row.columns = range(df_with_header_as_row.shape[1])
# df_with_header_as_row = df_with_header_as_row.astype(float)
print(df_with_header_as_row)
file_path = "E:/MedBank/head_model/head/work_dir/NGS_0/test/feature_vectors.csv"
df_with_header_as_row.to_csv(file_path, index=False, header=False)

          0         1         2         3     4         5         6         7  \
0  0.000000  0.009304  1.014189  0.586183  10.0  0.004652  0.000000  0.002326   
1  0.001754  0.005263  0.900000  0.722807  10.0  0.000000  0.000000  0.003509   
2  0.000000  0.021668  1.018418  0.167931  10.0  0.000000  0.005417  0.016251   
3  0.000000  0.012099  1.037508  0.465820  10.0  0.000000  0.003025  0.000000   
4  0.000000  0.037096  2.331744  0.349762  10.0  0.000000  0.010599  0.000000   

          8         9  
0  0.000000  0.009304  
1  0.003509  0.003509  
2  0.000000  0.010834  
3  0.000000  0.009074  
4  0.000000  0.000000  


In [13]:
#Normalizing training data

df_train = pd.read_csv("E:/MedBank/head_model/head/work_dir/0/train/feature_vectors.csv", header=None)

scaler = MinMaxScaler()
df_train[:] = scaler.fit_transform(df_train)
# df_with_header_as_row = df_with_header_as_row.apply(lambda x: 10 * (x - x.min()) / (x.max() - x.min()), axis=1) 
print(df_train)
file_path = "E:/MedBank/head_model/head/work_dir/NGS_0/train/feature_vectors.csv"
df_train.to_csv(file_path, index=False, header=False)

             0         1         2         3         4         5         6   \
0      0.601375  0.579670  0.359686  0.006429  0.231284  0.637124  0.470694   
1      0.000000  0.528293  0.180715  0.150223  0.009908  0.298529  0.428670   
2      0.219714  0.564028  0.435180  0.164055  0.009908  0.481011  0.360547   
3      0.529921  0.440363  0.620699  0.006429  0.009908  0.673714  0.398049   
4      0.274445  0.695183  0.424996  0.132618  0.246404  0.619574  0.423359   
...         ...       ...       ...       ...       ...       ...       ...   
12947  0.537337  0.824209  0.000000  0.389004  0.009908  0.563534  0.767739   
12948  0.692802  0.560395  0.517262  0.265034  0.354548  0.650092  0.532357   
12949  0.410751  0.423000  0.408968  0.006429  0.009908  0.237621  0.308724   
12950  0.000000  0.476363  0.409141  0.006429  0.425601  0.238650  0.209455   
12951  0.660770  0.643247  0.493239  0.177752  0.250985  0.670600  0.501749   

             7         8         9         10      

In [25]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])
# df = df.drop(df.columns[0], axis=1)
print('After normalization')
print(df)

After normalization
            gene_id   Sample1   Sample2   Sample3   Sample4   Sample5
0     hsa-let-7a-5p  0.352151  0.480096  0.273508  0.293495  0.188074
1     hsa-let-7a-3p  0.000005  0.000003  0.000049  0.000002  0.000006
3     hsa-let-7b-5p  0.092060  0.089210  0.059000  0.070440  0.073770
4     hsa-let-7b-3p  0.000016  0.000019  0.000016  0.000027  0.000016
5     hsa-let-7c-5p  0.001784  0.002031  0.004146  0.001976  0.001140
...             ...       ...       ...       ...       ...       ...
2697  hsa-miR-12126  0.000002  0.000000  0.000000  0.000000  0.000000
2698  hsa-miR-12127  0.000035  0.000049  0.000139  0.000039  0.000018
2701  hsa-miR-12130  0.000002  0.000000  0.000000  0.000000  0.000000
2705  hsa-miR-12135  0.000136  0.000289  0.000590  0.000118  0.000075
2706  hsa-miR-12136  0.000300  0.000643  0.000344  0.000376  0.000176

[1652 rows x 6 columns]


C:\Users\91786\AppData\Local\Temp\ipykernel_22356\280294872.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3.52150922e-01 4.61027076e-06 9.20601917e-02 ... 2.30513538e-06
 1.36002987e-04 2.99667599e-04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])
C:\Users\91786\AppData\Local\Temp\ipykernel_22356\280294872.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4.80096310e-01 3.24931683e-06 8.92099936e-02 ... 0.00000000e+00
 2.89189198e-04 6.43364733e-04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])
C:\Users\91786\AppData\Local\Temp\ipykernel_22356\280294872.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in

In [29]:
df2 = df.T
df2.columns = df2.iloc[0]
df2 = df2[1:]


In [30]:
df2.columns = range(df2.shape[1])
# print(df2)
df2.reset_index(drop=True, inplace=True)



             0         1        2         3         4         5         6     \
Sample1  0.352151  0.000005  0.09206  0.000016  0.001784   0.10714  0.000526   
Sample2  0.480096  0.000003  0.08921  0.000019  0.002031  0.144533  0.000601   
Sample3  0.273508  0.000049    0.059  0.000016  0.004146  0.096359  0.000574   
Sample4  0.293495  0.000002  0.07044  0.000027  0.001976  0.091928  0.000425   
Sample5  0.188074  0.000006  0.07377  0.000016   0.00114  0.048142  0.000294   

             7         8         9     ...      1642      1643      1644  \
Sample1  0.011406  0.000002  0.331419  ...       0.0  0.000007       0.0   
Sample2  0.009163       0.0  0.482719  ...       0.0  0.000003  0.000003   
Sample3  0.006556       0.0  0.254136  ...  0.000008  0.000008  0.000008   
Sample4  0.013394  0.000002  0.273512  ...  0.000002  0.000007       0.0   
Sample5  0.004712       0.0  0.154514  ...  0.000002  0.000018       0.0   

             1645      1646      1647      1648      1649     

In [112]:
file_path = "E:/MedBank/head_model/NGS/NGS_0/test/feature_vectors.csv"
df2.to_csv(file_path, index=False, header=False)


In [109]:
labels

[0, 0, 0, 0, 0]

In [113]:
file_path = "E:/MedBank/head_model/NGS/NGS_0/test/labels.txt"

with open(file_path, 'w') as file:
    for label in labels:
        file.write(f"{label}\n")


In [114]:
file_path = "E:/MedBank/head_model/NGS/NGS_0/label_names.txt"

with open(file_path, 'w') as file:
    file.write('sample_Label')

python single_estimator.py --input_dir work_dir/NGS_0 --result_dir work_dir/NGS_results_0 --seed 42 --estimator random_forest --n_samples -1 --n_features -1